In [38]:
import pandas as pd
import numpy as np
import spacy
import unicodedata
import re

In [42]:
df = pd.read_csv("dataset_nlp_clean.csv", encoding='utf8')
df2 = df[['nlp']]
df2['nlp'] = df2['nlp'].replace(np.NaN, ' ')
df['Descripción del Anuncio'] = df['Descripción del Anuncio'].replace(np.NaN, ' ')

In [8]:
df2.head()

,nlp
0,mejoramiento de la iluminacion de la iglesia s...
1,adquisicion de uniformes para la direccion de ...
2,proyecto ejecutivo jardin vecinal colonia las ...
3,construccion de deportivo bicentenario
4,acondicionamiento de centro cultural y deportivo


In [32]:
def avg_word(sentence):
    words = sentence.split()
    if len(words) > 1:
        return (sum(len(word) for word in words)/len(words))
    else:
        return 0

In [39]:
nlp = spacy.load("es_core_news_lg", disable=['ner', 'parser'])
SpacyStopwords = nlp.Defaults.stop_words

# StopwordsOriginal = SpacyStopwords.words("spanish") # stopwords de spacy o nltk cuales son mejor?
StopwordsSP = []

for Word in SpacyStopwords:
    Stop = Word.lower()
    Stop_NFKD = unicodedata.normalize('NFKD', Stop)
    Stop = u"".join([c for c in Stop_NFKD if not unicodedata.combining(c)])
    StopwordsSP.append(Stop)

preposiciones = ['a', 'durante', 'según', 'ante', 'en', 'sin', 'bajo', 'entre', 'so', 'cabe', 'hacia', 'sobre', 'con', 'hasta', 'tras', 'contra', 'mediante', 'versus', 'de', 'para', 'via', 'desde', 'por', 'y', 'o']

StopwordsSP = StopwordsSP + preposiciones

In [43]:
df2['word_count'] = df2['nlp'].apply(lambda x: len(str(x).split(" ")))
df2['char_count_w_spaces'] = df2['nlp'].str.len()
df2['char_count_wo_spaces'] = df2['nlp'].replace(" ", "", regex=True).str.len()
df2['avg_word'] = df2['nlp'].apply(lambda x: avg_word(x))
df2['stopwords'] = df2['nlp'].apply(lambda x: len([x for x in x.split() if x in StopwordsSP]))
# df2['hastags'] = df2['nlp'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df2['numerics'] = df2['nlp'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df2['punctuation'] = df['Descripción del Anuncio'].str.findall(r'[^a-zA-Z0-9 ]').str.len()
df2['upper'] = df['Descripción del Anuncio'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [44]:
df2.head()

,nlp,word_count,char_count_w_spaces,char_count_wo_spaces,avg_word,stopwords,numerics,punctuation,upper
0,mejoramiento de la iluminacion de la iglesia s...,9,55,47,5.222222,4,0,1,0
1,adquisicion de uniformes para la direccion de ...,11,74,64,5.818182,5,0,3,0
2,proyecto ejecutivo jardin vecinal colonia las ...,13,87,75,5.769231,4,0,1,13
3,construccion de deportivo bicentenario,4,38,35,8.750000,1,0,0,4
4,acondicionamiento de centro cultural y deportivo,6,48,43,7.166667,2,0,0,6


In [45]:
df2.to_csv("dataset_nlp_features.csv", index=False)